In [1]:
import sys
!{sys.executable} -m pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 11.8 MB/s eta 0:00:00


In [2]:
import sys
!{sys.executable} -m pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 4.6 MB/s eta 0:00:00


In [3]:
import sys
!{sys.executable} -m pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 7.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=b59f9b36ed28814665453cdffe871223fbb124152901523925fad94817a3a13b
  Stored in directory: /home/idies/.cache/pip/wheels/20/7c/09/4ad42725a29fce4bc21137c7f25f062b3655a4aea5b0e8d9a2
Successfully built pathtools
  Attempting uninstall: GitPython
    Found existing installation: GitPython 3.1.29
    Uninstalling GitPython-3.1.29:
      Successfully uninstalled GitPython-3.1.29


In [4]:
import os
os.environ['WANDB_API_KEY'] = 'b3c6dfb61d66930ee7d291c6b19c0ae0b5007c62'

In [5]:
import wandb
wandb.init(project="exoplanetml", entity="sakhawat",save_code=True,name='AdaBoost Classifier',job_type='analysis',reinit=True)

wandb: Currently logged in as: sakhawat. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.8
wandb: Run data is saved locally in /home/idies/workspace/Temporary/s.hossain18/scratch/jobs/20230802/20230802232202-245562/wandb/run-20230802_232314-49yjb8iz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run AdaBoost Classifier
wandb: ⭐️ View project at https://wandb.ai/sakhawat/exoplanetml
wandb: 🚀 View run at https://wandb.ai/sakhawat/exoplanetml/runs/49yjb8iz


In [6]:
import numpy as np

In [7]:
X_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_train_pca.npy')
X_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_test_pca.npy')
y_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_train.npy')
y_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_test.npy')

In [8]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=41)

In [9]:
from skopt import BayesSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [10]:
# Create an instance of the decision tree classifier with a specified maximum depth
base_estimator = DecisionTreeClassifier(max_depth=3, random_state=11)

In [11]:
# Create an instance of the classifier
adaboost_classifier = AdaBoostClassifier(estimator=base_estimator,random_state=43)

In [12]:
# Define the parameter grid
params = {
    'n_estimators': (200, 500),
    'learning_rate': (0.01, 0.5),
    'algorithm': ['SAMME', 'SAMME.R']
}

In [13]:
# Log the hyperparameters to wandb.config
wandb.config.update(params)

In [14]:
# Create an instance of BayesianSearchCV
bayes_cv = BayesSearchCV(adaboost_classifier, params, n_iter=20, cv=cv, n_jobs=-1)

In [15]:
# Perform hyperparameter tuning using Bayesian optimization
bayes_cv.fit(X_train, np.ravel(y_train))

BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=10, random_state=41),
              estimator=AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3,
                                                                            random_state=11),
                                           random_state=43),
              n_iter=20, n_jobs=-1,
              search_spaces={'algorithm': ['SAMME', 'SAMME.R'],
                             'learning_rate': (0.01, 0.5),
                             'n_estimators': (200, 500)})

In [16]:
# Print the best score and hyperparameters
print("Train score:", bayes_cv.best_score_)
print("Test score:", bayes_cv.score(X_test, y_test))
print("Best hyperparameters:", bayes_cv.best_params_)

Train score: 0.7981780055917987
Test score: 0.8020679468242246
Best hyperparameters: OrderedDict([('algorithm', 'SAMME'), ('learning_rate', 0.5), ('n_estimators', 500)])


In [17]:
# Log the best score and hyperparameters
wandb.log({"Train score": bayes_cv.best_score_})
wandb.log({"Test score": bayes_cv.score(X_test, y_test)})
wandb.log({"Best hyperparameters": bayes_cv.best_params_})

In [18]:
wandb.finish()

wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:  Test score ▁
wandb: Train score ▁
wandb: 
wandb: Run summary:
wandb:  Test score 0.80207
wandb: Train score 0.79818
wandb: 
wandb: 🚀 View run AdaBoost Classifier at: https://wandb.ai/sakhawat/exoplanetml/runs/49yjb8iz
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230802_232314-49yjb8iz/logs
